## Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import STOPWORDS,WordCloud
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
import pandas as pd
from pprint import pprint
import string
import os
import re
import numpy as np 
import pandas as pd 
import re
import nltk 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
import joblib
import seaborn as sns

### 1. Data Loading

In [ ]:
class DataLoader:
  def __init__(self,dir_name,file_name):
    self.dir_name = dir_name
    self.file_name = file_name
    
 
  def read_csv(self):
    os.chdir(self.dir_name)
    tweets_df=pd.read_csv(self.file_name)
    return tweets_df

data_loader_object = DataLoader('/content/drive/MyDrive/', 'Week1_challenge_data_source(CSV).csv')
raw_data = data_loader_object.read_csv()

##### Task 4. 1 - Write a python program to assign:
1. engagement score to each user. Consider the engagement score as the Euclidean distance between the user data point & the less engaged cluster (use the first clustering for this) (Euclidean Distance)
2. experience score to each user. Consider the experience score as the Euclidean distance between the user data point & the worst experience’s cluster. 


###### Engagment score

In [ ]:
# 1 - First get the engagment centroids and the given data
engagement_centroids = pd.read_csv("engagment_centroids.csv")
engagement_data = pd.read_csv("engagment_data.csv")

In [ ]:
# the first clusture is least engeged, so we will take that centroid as refrence
ref_centroid_eng = engagement_centroids[:1].to_numpy()

In [ ]:
# Calculate the euclidean distance as an engagement score
from sklearn.metrics.pairwise import euclidean_distances
enagement_score = euclidean_distances(engagement_data.to_numpy(), ref_centroid_eng)

###### Experience Score

In [ ]:
# 1 - First get the experience centroids and the given data
experience_centroids = pd.read_csv("experience_centriods.csv")
experience_data = pd.read_csv("experience_data.csv")

In [ ]:
# the first clusture is least experienced, so we will take that centroid as refrence
ref_centroid_exp = experience_centroids[:1].to_numpy()

In [ ]:
# Calculate the euclidean distance as an engagement score
from sklearn.metrics.pairwise import euclidean_distances
experience_score = euclidean_distances(experience_data.to_numpy(), ref_centroid_exp) 

In [ ]:
enagement_score

array([[0.0834707 ],
       [0.03914338],
       [0.03593144],
       ...,
       [0.026841  ],
       [0.03935664],
       [0.04986459]])

##### Task 4.2 - Consider the average of both engagement & experience scores as  the satisfaction score & report the top 10 satisfied customer 


In [ ]:
sat_sc = np.mean( np.array([ experience_score, enagement_score]), axis=0 )

In [ ]:
final_data = pd.DataFrame(sat_sc, columns=["Satisfaction Score"])

In [ ]:
final_data.nlargest(n=10, columns=["Satisfaction Score"])["Satisfaction Score"]

In [ ]:
final_columns = ["MSISDN/Number", "Average RTT", "Average TCP retransmission", 
                 "Average throughput", "Frequency", "DL Duration", "UL Duration",
                 "Sessions Total UL", "Sessions Total", "Satisfaction Score"]
Final_Data = pd.DataFrame(columns=final_columns)

In [ ]:
Final_Data["MSISDN/Number"] = raw_data.groupby("MSISDN/Number").agg({"Bearer Id": "count"}).reset_index()["MSISDN/Number"]
Final_Data["Average RTT"] = experience_data["0"]
Final_Data["Average TCP retransmission"] = experience_data["1"]
Final_Data["Average throughput"] = experience_data["2"]
Final_Data["Frequency"] = engagement_data["0"]
Final_Data["DL Duration"] = engagement_data["1"]
Final_Data["UL Duration"] = engagement_data["2"]
Final_Data["Sessions Total UL"] = engagement_data["3"]
Final_Data["Sessions Total"] = engagement_data["4"]
Final_Data["Satisfaction Score"] = final_data["Satisfaction Score"]


##### Task 4.3 - Build a regression model of your choice to predict the satisfaction score of a customer. 

In [ ]:
Final_Data.drop(["Satisfaction Score"], axis=1)

In [ ]:
x = Final_Data.drop(["MSISDN/Number", "Satisfaction Score"], axis=1)
y = Final_Data["Satisfaction Score"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.svm import SVR

svm_poly_reg = SVR(kernel="poly", degree=4, C=100, epsilon=0.1, gamma="scale")
svm_poly_reg.fit(x_train, y_train)

SVR(C=100, degree=4, kernel='poly')

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = svm_poly_reg.predict(x_train)
mse = mean_squared_error(y_train, y_pred)
mse

0.003953587781627983

#####  Run a k-means (k=2) on the engagement & the experience score .

In [159]:
from sklearn.preprocessing import MinMaxScaler
def normalizer(df):
  norm = MinMaxScaler()
  # normalize the exponential data with boxcox
  normalized_data = norm.fit_transform(df)
  return normalized_data

engament_list = [score[0] for score in enagement_score]
experience_list = [score[0] for score in experience_score]

en_ex_score = pd.DataFrame()

In [163]:
en_ex_score["engagement score"] = engament_list
en_ex_score["experience score"] = experience_list

en_ex_norm = normalizer(en_ex_score)

In [164]:
from sklearn.cluster import KMeans
k = 3
kmeans = KMeans(n_clusters=k, random_state=42)
y_pred = kmeans.fit_predict(en_ex_norm)

In [165]:
en_ex_score["cluster"] = y_pred

In [168]:
en_ex_score = en_ex_score.sort_values(by=["cluster"])

##### Task 4.6 - Export your final table containing all user id + engagement, experience & satisfaction scores in your local MySQL database. Report a screenshot of a select query output on the exported table. 


In [169]:
saved_data = pd.DataFrame(columns=["ID", "Engagement Score", 
                                   "Experience Score", "Satisfaction Score"])
saved_data["ID"] = Final_Data["MSISDN/Number"]
saved_data["Engagement Score"] = engament_list
saved_data["Experience Score"] = experience_list
saved_data["Satisfaction Score"] = Final_Data["Satisfaction Score"]


In [170]:
saved_data.to_csv("final_data.csv", index=False)